In [ ]:
!pip install -r https://raw.githubusercontent.com/huggingface/deep-rl-class/main/notebooks/unit2/requirements-unit2.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.1/132.1 kB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 682.2/682.2 kB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 41.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 45.2 MB/s eta 0:00:00
  Created wheel for pickle5: filename=pickle5-0.0.11-cp310-cp310-linux_x86_64.whl size=255320 sha256=57ac82542d2fb9e718f22bc13b45b11ad59a3dc8174ba6b5a41dde7d46fb29a0
  Stored in directory: /root/.cache/pip/wheels/7d/14/ef/4aab19d27fa8e58772be5c71c16add0426acf9e1f64353235c
Successfully built pickle5
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 6.0.2
    Uninstalling PyYAML-6.0.2:
      Successfully uninstalled PyYAML-6.0.2


In [ ]:
!sudo apt-get update
!sudo apt-get install -y python3-opengl
!apt install ffmpeg xvfb
!pip3 install pyvirtualdisplay

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Ign:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:5 https://r2u.stat.illinois.edu/ubuntu jammy Release
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [921 kB]
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,152 kB]
Hit:14 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:

In [ ]:
import os
os.kill(os.getpid(), 9)

In [ ]:
from pyvirtualdisplay import Display

virtual_display = Display(visible=0, size=(1400, 900))
virtual_display.start()

In [ ]:
import numpy as np
import gymnasium as gym
import random
import imageio
import os
import tqdm

import pickle5 as pickle
from tqdm.notebook import tqdm

# Part1: Frozen Lake (NonSlippery Version)

In [ ]:
env = gym.make('FrozenLake-v1', map_name='4x4', is_slippery=False, render_mode='rgb_array')

In [ ]:
env.desc

/usr/local/lib/python3.10/dist-packages/gymnasium/core.py:311: UserWarning: WARN: env.desc to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.desc` for environment variables or `env.get_wrapper_attr('desc')` that will search the reminding wrappers.
  logger.warn(


array([[b'S', b'F', b'F', b'F'],
       [b'F', b'H', b'F', b'H'],
       [b'F', b'F', b'F', b'H'],
       [b'H', b'F', b'F', b'G']], dtype='|S1')

In [ ]:
desc = ['SFFF', 'FHFH', 'FFFH', 'HFFG']
env_custom = gym.make('FrozenLake-v1', desc=desc, is_slippery=True)
env_custom.desc

array([[b'S', b'F', b'F', b'F'],
       [b'F', b'H', b'F', b'H'],
       [b'F', b'F', b'F', b'H'],
       [b'H', b'F', b'F', b'G']], dtype='|S1')

In [ ]:
print('___Observation Space___ \n')
print('Observation Space: ', env.observation_space)
print('Sample Observation: ', env.observation_space.sample())

___Observation Space___ 

Observation Space:  Discrete(16)
Sample Observation:  2


In [ ]:
print('___Action Space___ \n')
print('Action Space Shape: ', env.action_space.n)
print('Action Space Sample: ', env.action_space.sample())

___Action Space___ 

Action Space Shape:  4
Action Space Sample:  0


In [ ]:
state_space = env.observation_space.n
print(f'There are {state_space} possible states')

action_space = env.action_space.n
print(f'There are {action_space} possible actions')

There are 16 possible states
There are 4 possible actions


In [ ]:
def initialize_q_table(state_space, action_space):
    Qtable = np.zeros((state_space, action_space))
    return Qtable

In [ ]:
Qtable_frozenlake = initialize_q_table(state_space, action_space)
Qtable_frozenlake.shape

(16, 4)

In [ ]:
def greedy_policy(Qtable, state):
    action = np.argmax(Qtable[state][:])

    return action

In [ ]:
def epsilon_greedy_policy(Qtable, state, epsilon):
    random_num = random.uniform(0, 1)

    if random_num > epsilon:
        action = greedy_policy(Qtable, state)
    else:
        action = env.action_space.sample()

    return action

In [ ]:
n_training_episodes = 10000
learning_rate = 0.7

n_eval_episodes = 1000

env_id = 'FrozenLake-v1'
max_steps = 99
gamma = 0.95
eval_seed = []

max_epsilon = 1.0
min_epsilon = 0.05
decay_rate = 0.0005

In [ ]:
def train(n_training_episodes, min_epsilon, max_epsilon, decay_rate, env, max_steps, Qtable):
    for episode in tqdm(range(n_training_episodes)):
        epsilon = min_epsilon + (max_epsilon - min_epsilon) * np.exp(-decay_rate * episode)

        state, info = env.reset()
        step = 0
        terminated = False
        truncated = False

        for step in range(max_steps):
            action = epsilon_greedy_policy(Qtable, state, epsilon)
            new_state, reward, terminated, truncated, info = env.step(action)

            Qtable[state][action] = Qtable[state][action] + learning_rate * (reward + gamma * np.max(Qtable[new_state]) - Qtable[state][action])

            if terminated or truncated:
                break

            state = new_state
    return Qtable

In [ ]:
Qtable_frozenlake = train(n_training_episodes, min_epsilon, max_epsilon, decay_rate, env, max_steps, Qtable_frozenlake)

  0%|          | 0/10000 [00:00<?, ?it/s]

In [ ]:
Qtable_frozenlake.shape

(16, 4)

In [ ]:
Qtable_frozenlake

array([[0.73509189, 0.77378094, 0.77378094, 0.73509189],
       [0.73509189, 0.        , 0.81450625, 0.77378094],
       [0.77378094, 0.857375  , 0.77378094, 0.81450625],
       [0.81450625, 0.        , 0.77378094, 0.77378094],
       [0.77378094, 0.81450625, 0.        , 0.73509189],
       [0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.9025    , 0.        , 0.81450625],
       [0.        , 0.        , 0.        , 0.        ],
       [0.81450625, 0.        , 0.857375  , 0.77378094],
       [0.81450625, 0.9025    , 0.9025    , 0.        ],
       [0.857375  , 0.95      , 0.        , 0.857375  ],
       [0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.9025    , 0.95      , 0.857375  ],
       [0.9025    , 0.95      , 1.        , 0.9025    ],
       [0.        , 0.        , 0.        , 0.        ]])

In [ ]:
def evaluate_agent(env, max_steps, n_eval_episodes, Q, seed):
    episode_rewards = []
    for episode in tqdm(range(n_eval_episodes)):
        if seed:
            state, info = env.reset(seed=seed[episode])
        else:
            state, info = env.reset()
        step = 0
        truncated = False
        terminated = False
        total_rewards_ep = 0

        for step in range(max_steps):
            action = greedy_policy(Q, state)
            new_state, reward, terminated, truncated, info = env.step(action)
            total_rewards_ep += reward

            if terminated or truncated:
                break
            state = new_state
        episode_rewards.append(total_rewards_ep)
    mean_reward = np.mean(episode_rewards)
    std_reward = np.std(episode_rewards)

    return mean_reward, std_reward

In [ ]:
mean_reward, std_reward = evaluate_agent(env, max_steps, n_eval_episodes, Qtable_frozenlake, eval_seed)
print(f"Mean_reward={mean_reward:.2f} +/- {std_reward:.2f}")

  0%|          | 0/1000 [00:00<?, ?it/s]

Mean_reward=1.00 +/- 0.00


In [ ]:
from huggingface_hub import HfApi, snapshot_download
from huggingface_hub.repocard import metadata_eval_result, metadata_save

from pathlib import Path
import datetime
import json

In [ ]:
def record_video(env, Qtable, out_directory, fps=1):
    images = []
    terminated = False
    truncated = False
    state, info = env.reset(seed=random.randint(0, 500))
    img = env.render()
    images.append(img)
    while not terminated or truncated:
        action = np.argmax(Qtable[state][:])
        state, reward, terminated, truncated, info = env.step(action)
        img = env.render()
        images.append(img)
    imageio.mimsave(out_directory, [np.array(img) for i, img in enumerate(images)], fps=fps)

In [ ]:
def push_to_hub(repo_id, model, env, video_fps=1, local_repo_path='hub'):
    _, repo_name = repo_id.split("/")

    eval_env = env
    api = HfApi()

    repo_url = api.create_repo(
        repo_id=repo_id,
        exist_ok=True,
    )

    repo_local_path = Path(snapshot_download(repo_id=repo_id))

    if env.spec.kwargs.get('map_name'):
        model['map_name'] = env.spec.kwargs.get('map_name')
        if env.spec.kwargs.get('is_slippery', '') == False:
            model['slippery'] = False

    with open((repo_local_path) / 'q-learning.pkl', 'wb') as f:
        pickle.dump(model, f)

    mean_reward, std_reward = evaluate_agent(
        eval_env, model['max_steps'], model['n_eval_episodes'], model['qtable'], model['eval_seed']
    )

    evaluate_data = {
        'env_id': model['env_id'],
        'mean_reward': mean_reward,
        'n_eval_episodes': model['n_eval_episodes'],
        'eval_datetime': datetime.datetime.now().isoformat(),
    }

    with open(repo_local_path / 'results.json', 'w') as outfile:
        json.dump(evaluate_data, outfile)

    env_name = model['env_id']
    if env.spec.kwargs.get('map_name'):
        env_name += '-' + env.spec.kwargs.get('map_name')

    if env.spec.kwargs.get('is_slippery', '') == False:
        env_name += '-' + 'no_slippery'

    metadata = {}
    metadata["tags"] = [env_name, 'q-learning', 'reinforcement-learning', 'custom-implementation']

    eval = metadata_eval_result(
        model_pretty_name=repo_name,
        task_pretty_name='reinforcement-learning',
        task_id='reinforcement-learning',
        metrics_pretty_name='mean_reward',
        metrics_id='mean_reward',
        metrics_value=f"{mean_reward:.2f} +/- {std_reward:.2f}",
        dataset_pretty_name=env_name,
        dataset_id=env_name,
    )

    metadata = {**metadata, **eval}

    model_card = f"""
  # **Q-Learning** Agent playing1 **{env_id}**
  This is a trained model of a **Q-Learning** agent playing **{env_id}** .

  ## Usage

  model = load_from_hub(repo_id="{repo_id}", filename="q-learning.pkl")

  # Don't forget to check if you need to add additional attributes (is_slippery=False etc)
  env = gym.make(model["env_id"])
  """

    evaluate_agent(env, model['max_steps'], model['n_eval_episodes'], model['qtable'], model['eval_seed'])

    readme_path = repo_local_path / 'README.md'
    readme = ''
    print(readme_path.exists())
    if readme_path.exists():
        with readme_path.open('r', encoding='utf8') as f:
            readme = f.read()
    else:
        readme = model_card

    with readme_path.open('w', encoding='utf-8') as f:
        f.write(readme)

    metadata_save(readme_path, metadata)

    video_path = repo_local_path / 'replay.mp4'
    record_video(env, model['qtable'], video_path, video_fps)

    api.upload_folder(
        repo_id=repo_id,
        folder_path=repo_local_path,
        path_in_repo='.',
    )

    print(f'Your model is pushed to the Hub. You can view your model here: {repo_url}')

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
model = {
    'env_id': env_id,
    'max_steps': max_steps,
    'n_training_episodes': n_training_episodes,
    'n_eval_episodes': n_eval_episodes,
    'eval_seed': eval_seed,
    'learning_rate': learning_rate,
    'gamma': gamma,
    'max_epsilon': max_epsilon,
    'min_epsilon': min_epsilon,
    'decay_rate': decay_rate,
    'qtable': Qtable_frozenlake
}

In [ ]:
username = 'khadivi-ah'
repo_name = 'FrozenLake-v1-4x4-no_slippery'
push_to_hub(repo_id=f'{username}/{repo_name}', model=model, env=env)

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

replay.mp4:   0%|          | 0.00/31.1k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/834 [00:00<?, ?B/s]

results.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

q-learning.pkl:   0%|          | 0.00/915 [00:00<?, ?B/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

/usr/lib/python3.10/subprocess.py:1796: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = _posixsubprocess.fork_exec(


# Part2: Taxi-v3

In [ ]:
env = gym.make('Taxi-v3', render_mode='rgb_array')

In [ ]:
state_space = env.observation_space.n
print(f'There are {state_space} possible states')

action_space = env.action_space.n
print(f'There are {action_space} possible actions')

In [ ]:
Qtable_taxi = initialize_q_table(state_space, action_space)
print(Qtable_taxi)

In [ ]:
Qtable_taxi.shape

(500, 6)

In [ ]:
n_training_episodes = 25000
learning_rate = 0.7

n_eval_episodes = 100

eval_seed = [
      16,
    54,
    165,
    177,
    191,
    191,
    120,
    80,
    149,
    178,
    48,
    38,
    6,
    125,
    174,
    73,
    50,
    172,
    100,
    148,
    146,
    6,
    25,
    40,
    68,
    148,
    49,
    167,
    9,
    97,
    164,
    176,
    61,
    7,
    54,
    55,
    161,
    131,
    184,
    51,
    170,
    12,
    120,
    113,
    95,
    126,
    51,
    98,
    36,
    135,
    54,
    82,
    45,
    95,
    89,
    59,
    95,
    124,
    9,
    113,
    58,
    85,
    51,
    134,
    121,
    169,
    105,
    21,
    30,
    11,
    50,
    65,
    12,
    43,
    82,
    145,
    152,
    97,
    106,
    55,
    31,
    85,
    38,
    112,
    102,
    168,
    123,
    97,
    21,
    83,
    158,
    26,
    80,
    63,
    5,
    81,
    32,
    11,
    28,
    148
]

env_id = 'Taxi-v3'
max_steps = 99
gamma = 0.95

max_epsilon = 1.0
min_epsilon = 0.05
decay_rate = 0.005

In [ ]:
Qtable_taxi = train(n_training_episodes, min_epsilon, max_epsilon, decay_rate, env, max_steps, Qtable_taxi)

  0%|          | 0/25000 [00:00<?, ?it/s]

In [ ]:
Qtable_taxi

array([[  0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ],
       [  2.75200369,   3.94947723,   2.75200369,   3.94947757,
          5.20997639,  -5.05052245],
       [  7.93349184,   9.40367562,   7.93349184,   9.40367544,
         10.9512375 ,   0.40367562],
       ...,
       [ -2.94616347,  12.58024968,  -3.37307717,  -2.95958523,
         -7.        , -11.69011595],
       [ -4.70520633,  -4.6423991 ,  -4.18576064,   6.53681725,
        -11.77905681, -10.19366897],
       [ -1.3755    ,  12.79277776,  -1.3755    ,  18.        ,
         -9.5655    ,   5.64190983]])

In [ ]:
mean_reward, std_reward = evaluate_agent(env, max_steps, n_eval_episodes, Qtable_taxi, eval_seed)
print(f"Mean_reward={mean_reward:.2f} +/- {std_reward:.2f}")

  0%|          | 0/100 [00:00<?, ?it/s]

In [ ]:
model = {
    'env_id': env_id,
    'max_steps': max_steps,
    'n_training_episodes': n_training_episodes,
    'n_eval_episodes': n_eval_episodes,
    'eval_seed': eval_seed,
    'learning_rate': learning_rate,
    'gamma': gamma,
    'max_epsilon': max_epsilon,
    'min_epsilon': min_epsilon,
    'decay_rate': decay_rate,
    'qtable': Qtable_taxi
}

In [ ]:
username = 'khadivi-ah'
repo_name = 'Taxi-v3'
push_to_hub(repo_id=f'{username}/{repo_name}', model=model, env=env)

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

/usr/lib/python3.10/subprocess.py:1796: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = _posixsubprocess.fork_exec(


q-learning.pkl:   0%|          | 0.00/24.6k [00:00<?, ?B/s]

In [ ]:
from urllib.error import HTTPError
from huggingface_hub import hf_hub_download

In [ ]:
def load_from_hub(repo_id: str, filename: str) -> str:
  pickle_model = hf_hub_download(repo_id=repo_id, filename=filename)

  with open(pickle_model, 'rb') as file:
    downloaded_model_file = pickle.load(file)

  return downloaded_model_file

In [ ]:
model = load_from_hub(repo_id='khadivi-ah/Taxi-v3', filename='q-learning.pkl')

q-learning.pkl:   0%|          | 0.00/24.6k [00:00<?, ?B/s]

In [ ]:
env = gym.make(model['env_id'])
evaluate_agent(env, model['max_steps'], model['n_eval_episodes'], model['qtable'], model['eval_seed'])

  0%|          | 0/100 [00:00<?, ?it/s]

(7.56, 2.706732347314747)